In [1]:
#importing the dependencies
import pandas as pd
import numpy as np
import time
import datetime
import os
from datetime import datetime as dt
from glob import glob


# Set display to max for rows and columns
pd.set_option('display.max_rows', None)
pd.set_option("display.max_columns", None)

In [27]:
user_behavior_data = pd.read_csv(r'D:\Shyam\Documents\MUSER\Analysis\Cleaned Data\muser-user-master.csv',encoding='utf-8-sig')

In [28]:
# 1.0
# Create a dataframe to Group By features
# This one is grouped by User_ID then Song Features
user_song_group = user_behavior_data.groupby(["user_id","song_artist_name","song_album_name", "song_name"])

In [29]:
# Function to calculate feature metrics
def my_agg(x):
    names = {
        'Elapsed Time Sum': x['elapsed_time_in_min'].sum(),
        'Half-Song Listening Time': x['player_half_song'].sum(),
        'Count of Songs': x['song_name'].count(),#nunique()
        'Count of Half-Songs': x['player_half_song'].nunique(),
        'Start Date': x['date'].min(),
        'End Date': x['date'].max(),
        'Date Unique Count': x['date'].nunique()}

    return pd.Series(names, index=['Elapsed Time Sum', 'Half-Song Listening Time', 'Count of Songs','Count of Half-Songs','Start Date', 'End Date', 'Date Unique Count'])

user_song_group.apply(my_agg)

Elapsed Time Sum  \
user_id                      song_artist_name                                   song_album_name                                    song_name                                                              
2s6FQ5Y3ETTr0R2rAgAT48qrm3H3 Adele                                              25                                                 Send My Love (To Your New Lover)                              1.3308   
                             Jackie Evancho                                     Dream With Me                                      When You Wish Upon A Star                                     0.2512   
                             Sir Thomas Beecham                                 Tropical Sweets                                    Daybreak from the Florida Suite                               0.5577   
                             Whitney Houston                                    Live: Her Greatest Performances                    You Give Good Love [Live from The Tonight Show ...            3.0240   
8LJfpayhQZVGDA27hwbVkzNTRmJ2 Kids in Love                                       Kids in Love                                       Never Let You Go                                              0.0000   
                                                                                                                                   Remind Me to Forget                                           7.1266   
ADj7r0mSsPeqU0cPctnp0Gu1TMB2 Samsung                                            Brand Music 2020                                   Over the Horizon                                              2.5254   
AcXHUu8sT8eyNxMPkFAtfKOf4SB3 Jackie Evancho                                     Dream With Me                                      When You Wish Upon A Star                                     0.2041   
Amz50zcWvpYZiJdnJUcOXcb8az33 Adele                                              25                                                 I Miss You                                                    0.8081   
                                                                                                                                   Send My Love (To Your New Lover)                              0.7665   
                                                                                                                                   Water Under The Bridge                                        0.6451   
                             Allan Jones                                        Hits of the 30's & 40's, Vol. 1, Disc 2            The Donkey Serenade                                           0.0776   
                             Arthur Fiedler                                     Hits of the 30's & 40's, Vol. 1, Disc 2            Jalouise                                                      0.0688   
                             Artie Shaw & His Orchestra                         Hits of the 30's & 40's, Vol. 1 & 2, Disc 3        Frenesi                                                       0.0812   
                             Benny Goodman & His Orchestra                      Hits of the 30's & 40's, Vol. 1                    Goody Goody                                                   0.7664   
                                                                                Hits of the 30's & 40's, Vol. 1, Disc 2            And the Angels Sing                                           0.0896   
                             Bing Crosby                                        Hits of the 30's & 40's, Vol. 1, Disc 2            Please                                                        1.2081   
                             Bob Hope                                           Hits of the 30's & 40's, Vol. 1, Disc 2            Thanks for the Memory                                         0.1068   
                             Bunny Berigan                                      Hits of the 30's & 40's, Vol. 1, Disc 2    

In [30]:
# Save the grouped by results in a dataframe
user_song_group = user_song_group.apply(my_agg)

# Sort values by user_id ascending
user_behavior_data.sort_values('user_id', ascending=True, inplace=True, ignore_index = True)

In [31]:
# 2.0
# creating a second group by dataframe
# This one is grouped by User_Id and then Date followed by Songs Features
user_date_group = user_behavior_data.groupby(["user_id","date","song_name","song_artist_name","song_album_name"])

In [32]:
# Function to calculate feature metrics
def my_agg(x):
    names = {
        'Elapsed Time Sum': x['elapsed_time_in_min'].sum(),
        'Half-Song Listening Time': x['player_half_song'].sum(),
        'Count of Songs': x['song_name'].count(),#nunique()
        'Count of Half-Songs': x['player_half_song'].nunique()}
    
    return pd.Series(names, index=['Elapsed Time Sum', 'Half-Song Listening Time', 'Count of Songs','Count of Half-Songs'])

user_date_group.apply(my_agg)

Elapsed Time Sum  \
user_id                      date       song_name                                          song_artist_name                                   song_album_name                                                        
2s6FQ5Y3ETTr0R2rAgAT48qrm3H3 2021-07-26 Daybreak from the Florida Suite                    Sir Thomas Beecham                                 Tropical Sweets                                               0.5577   
                                        Send My Love (To Your New Lover)                   Adele                                              25                                                            1.3308   
                                        When You Wish Upon A Star                          Jackie Evancho                                     Dream With Me                                                 0.2512   
                                        You Give Good Love [Live from The Tonight Show ... Whitney Houston                                    Live: Her Greatest Performances                               3.0240   
8LJfpayhQZVGDA27hwbVkzNTRmJ2 2021-01-13 Never Let You Go                                   Kids in Love                                       Kids in Love                                                  0.0000   
                             2021-01-18 Remind Me to Forget                                Kids in Love                                       Kids in Love                                                  7.1266   
ADj7r0mSsPeqU0cPctnp0Gu1TMB2 2021-06-10 Over the Horizon                                   Samsung                                            Brand Music 2020                                              2.5254   
AcXHUu8sT8eyNxMPkFAtfKOf4SB3 2021-07-20 When You Wish Upon A Star                          Jackie Evancho                                     Dream With Me                                                 0.2041   
Amz50zcWvpYZiJdnJUcOXcb8az33 2021-07-12 A-Tisket, A-Tasket                                 Ella Fitzgerald                                    Hits of the 30's & 40's, Vol. 1                               4.2177   
                                        And the Angels Sing                                Benny Goodman & His Orchestra                      Hits of the 30's & 40's, Vol. 1, Disc 2                       0.0896   
                                        Boogie Woogie                                      Tommy Dorsey & His Orchestra                       Hits of the 30's & 40's, Vol. 1, Disc 2                       4.0084   
                                        Ciribiribin (They're So in Love)                   Harry James & His Orchestra                        Hits of the 30's & 40's, Vol. 1, Disc 2                       0.2126   
                                        Falling in Love Again                              Marlene Dietrich                                   Hits of the 30's & 40's, Vol. 1, Disc 2                       0.1091   
                                        Frenesi                                            Artie Shaw & His Orchestra                         Hits of the 30's & 40's, Vol. 1 & 2, Disc 3                   0.0812   
                                        I Can't Get Started                                Bunny Berigan                                      Hits of the 30's & 40's, Vol. 1, Disc 2                       0.0806   
                                        I Miss You                                         Adele                                              25                                                            0.8081   
                                        I'll Never Smile Again                             Tommy Dorsey & His Orchestra                       Hits of the 30's & 40's, Vol. 1 & 2, Disc 3                   0.0914   
                                        If I Didn't Care                                   Ink Spots, The                       

In [34]:
# Save the grouped by results in a dataframe
user_date_group = user_date_group.apply(my_agg)

# Sort values by user_id ascending
user_behavior_data.sort_values('user_id', ascending=True, inplace=True, ignore_index = True)

In [35]:
# CSV EXPORT
path = r'D:\Shyam\Documents\MUSER\Analysis\Cleaned Data'

# This is to export the first dataframe as a CSV
user_song_group.to_csv(os.path.join(path,'user_song_group_summary.csv'), index=True, encoding='utf-8-sig')

# This is to export the second dataframe as a separate CSV because it is formatted differently
user_date_group.to_csv(os.path.join(path,'user_date_group_summary.csv'), index=True, encoding='utf-8-sig')